In [2]:
import os
from binance.client import Client

In [3]:
print()
print(os.environ.get('binance_secret_key'))

oe4dViGH1VCOiigIeKCGJH498VxCXW6WnDe0Cw6mKK1elpZ1r4g9rXnfZWqOWTFr
MxbvEnxsv4ngTVBexHjyhRrsIbAEcaZW4epgNVmZvMw0RKImLqelOkRb6dwku6Mv


In [4]:
api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret_key')

In [11]:
client = Client(api_key, api_secret)
client.API_URL = 'https://testnet.binance.vision/api'

In [14]:
# get balances for all assets & some account information
print(client.get_account())

{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'updateTime': 1612334365704, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}


In [15]:
print(client.get_asset_balance(asset='BTC'))

{'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}


In [17]:
# get latest price from Binance API
btc_price = client.get_symbol_ticker(symbol="BTCUSDT")
# print full output (dictionary)
print(btc_price)
print(btc_price["price"])  # Dict

{'symbol': 'BTCUSDT', 'price': '772200.00000000'}
772200.00000000


In [19]:
import os

from binance.client import Client
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor

In [20]:
# init
api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')

client = Client(api_key, api_secret)
btc_price = {'error':False}

In [21]:
def btc_trade_history(msg):
    ''' define how to process incoming WebSocket messages '''
    if msg['e'] != 'error':
        print(msg['c'])
        btc_price['last'] = msg['c']
        btc_price['bid'] = msg['b']
        btc_price['last'] = msg['a']
    else:
        btc_price['error'] = True

In [24]:
# init and start the WebSocket
bsm = BinanceSocketManager(client)
conn_key = bsm.start_symbol_ticker_socket('BTCUSDT', btc_trade_history)
bsm.start()

35855.41000000
35855.41000000
35855.41000000
35855.41000000
35854.53000000
35854.53000000
35854.53000000


In [25]:
# stop websocket
bsm.stop_socket(conn_key)

# properly terminate WebSocket
reactor.stop()

35854.53000000


In [26]:
help(BinanceSocketManager)

Help on class BinanceSocketManager in module binance.websockets:

class BinanceSocketManager(threading.Thread)
 |  BinanceSocketManager(client, user_timeout=1800)
 |  
 |  A class that represents a thread of control.
 |  
 |  This class can be safely subclassed in a limited fashion. There are two ways
 |  to specify the activity: by passing a callable object to the constructor, or
 |  by overriding the run() method in a subclass.
 |  
 |  Method resolution order:
 |      BinanceSocketManager
 |      threading.Thread
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, client, user_timeout=1800)
 |      Initialise the BinanceSocketManager
 |      
 |      :param client: Binance API client
 |      :type client: binance.Client
 |      :param user_timeout: Custom websocket timeout
 |      :type user_timeout: int
 |  
 |  close(self)
 |      Close all connections
 |  
 |  run(self)
 |      Method representing the thread's activity.
 |      
 |      You may override

# Bitcoin’s historical price data in CSV format?

In [30]:
# valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

# get timestamp of earliest date data is available
timestamp = client._get_earliest_valid_timestamp('BTCUSDT', '1m')
print(timestamp)

1502942400000


In [31]:
# request historical candle (or klines) data
bars = client.get_historical_klines('BTCUSDT', '1d', timestamp, limit=1000)
# this function by default calls the last 500 points of data.

In [35]:
bars[0]

[1503014400000,
 '4285.08000000',
 '4371.52000000',
 '3938.77000000',
 '4108.37000000',
 '1199.88826400',
 1503100799999,
 '5086958.30617151',
 5233,
 '972.86871000',
 '4129123.31651808',
 '9384.14140858']

# fire an order for Ethereum using the Binance API

In [37]:
import os

from binance.client import Client
from binance.enums import *
from binance.exceptions import BinanceAPIException, BinanceOrderException

client = Client(api_key, api_secret)

In [43]:
buy_order_limit = client.create_order(symbol='ETHUSDT', side='BUY', type='LIMIT', timeInForce='GTC',
                                           quantity=100, price=200)

AttributeError: 'NoneType' object has no attribute 'encode'

In [41]:
buy_order = client.create_order(symbol='ETHUSDT', side='BUY', type='MARKET', quantity=10)

AttributeError: 'NoneType' object has no attribute 'encode'

In [40]:
# create a real order if the test orders did not raise an exception

try:
    buy_limit = client.create_order(
        symbol='ETHUSDT',
        side='BUY',
        type='LIMIT',
        timeInForce='GTC',
        quantity=100,
        price=200)

except BinanceAPIException as e:
    # error handling goes here
    print(e)
except BinanceOrderException as e:
    # error handling goes here
    print(e)

AttributeError: 'NoneType' object has no attribute 'encode'